## Please read the blog post, and codealong in this Jupyter notebook. 

https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

Please refer to "Recursion" from https://cr.yp.to/2005-261/bender2/DT.pdf , https://medium.com/greyatom/decision-trees-a-simple-way-to-visualize-a-decision-dc506a403aeb, https://www.ke.tu-darmstadt.de/lehre/archiv/ws0809/mldm/dt.pdf

### https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

In [1]:
# Imports

In [2]:
import numpy as np

In [13]:
class DecisionTreeClassifier():
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        '''
        Build the decision tree classifier.
        '''
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
        
    def _predict(self, inputs):
        '''
        Predict class for single sample
        '''
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class
    
    def _gini(self, y):
        """Compute Gini impurity of a non-empty node.
        Gini impurity is defined as Σ p(1-p) over all classes, with p the frequency of a
        class within the node. Since Σ p = 1, this is equivalent to 1 - Σ p^2.
        """
        m = y.size
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in range(self.n_classes_))
        
    def _grow_tree(self, X, y, depth=0):
        '''
        Build a decision tree by recursively finding the best split.
        '''
        # Population for each class in current node. 
        # The predicted class is the one with
        # largest population.
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        print(num_samples_per_class)
        predicted_class = np.argmax(num_samples_per_class)
        print(predicted_class)
        
        node = Node(
            gini=self._gini(y),
            num_samples=y.shape,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )
        
        # Split recursively until max_depth is reached
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                # using ~ will start counting index from the right instead of left
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
        
        
    def _best_split(self, X, y):
        """Find the best split for a node.
        "Best" means that the average impurity of the two children, weighted by their
        population, is the smallest possible. Additionally it must be less than the
        impurity of the current node.
        To find the best split, we loop through all the features, and consider all the
        midpoints between adjacent training samples as possible thresholds. We compute
        the Gini impurity of the split generated by that particular feature/threshold
        pair, and return the pair with smallest impurity.
        Returns:
            best_idx: Index of the feature for best split, or None if no split is found.
            best_thr: Threshold to use for the split, or None if no split is found.
        """
        
        m = y.size
        if m <= 1:
            return None, None
        
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        
        # Calculates Gini of current node
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        
        # Loop through all features.#
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                
                # Gini impurity calculated as the weighted average of the Gini
                # of the children
                gini  = (i * gini_left + (m - i) * gini_right) / m
                
                # The following condition is to make sure we don't try to
                # split two points with identical values for that feature,
                # as it is impossible (both have to end up on the same side
                # of a split).
                if thresholds[i] == thresholds[i - 1]:
                    continue
                    
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2 #midpoint

            return best_idx, best_thr
        
class Node():
    def __init__(
        self,
        gini,
        num_samples,
        num_samples_per_class,
        predicted_class,
    ):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.left = None
        self.right = None

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

In [19]:
# Load data.
df = pd.read_csv('wifi_localization.txt', delimiter='\t')
data = df.to_numpy()
X = data[:, :-1] # all columns but the last
y = data[:, -1] - 1 # expected to be from 0 to n_classes - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=101)

In [20]:
# Fit data.
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X_train, y_train)

# Visualize.
#clf.debug(
#    features_names=["Wifi {}".format(i) for i in range(1, 8)],
#    target_names=["Room {}".format(i) for i in range(1, 5)],
#)

[348, 362, 334, 355]
1
[348, 361, 334, 355]
1
[348, 357, 334, 355]
1
[0, 4, 0, 0]
1
[0, 1, 0, 0]
1


In [26]:
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

600

In [31]:
y_test.ravel()

array([354, 277, 405, 507, 407, 409, 505, 503, 502, 129, 130, 261, 132,
       493, 138, 139, 142, 252, 510, 424, 111, 280, 302,  86, 390, 395,
       294, 523, 286, 402, 519, 100, 101, 102, 207, 404, 282, 281, 513,
       109, 389, 486, 484, 441, 185, 219, 442, 459, 190, 444, 215, 194,
       212, 456, 455, 199, 449, 201, 451, 204, 463, 430, 465, 467, 482,
       156, 157, 239, 230, 166, 229, 169, 438, 171, 471, 173, 174, 226,
       225, 439, 222, 221, 304, 517, 387, 343,  26, 575, 364,  29, 574,
       340,  32, 344, 337, 332,  40,  42, 533, 372, 564, 327, 326,  34,
       578, 345,  21,   1, 596, 595,   5, 350,   7, 361,   9, 348, 362,
        12, 587, 586, 585, 584, 346,  18, 582, 581, 322,  50, 450,  58,
        52, 318, 379, 314, 380, 549,  68, 546, 310, 544,  73, 541, 552,
       384, 319,  60,  79,  53, 538, 377, 536, 278, 306, 316, 279, 208,
       301, 359, 382, 289, 385, 447, 309, 292, 291, 214, 400, 443, 399,
       223, 383, 376, 317, 412, 423, 266, 254, 419, 265, 269, 25

In [5]:
X = np.array([[4,5,6,3,4,5], [1,3,7,9,9,8]])
y = np.array([1,1,0,1,1,0])

# Fit data.
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X, y)

[2, 4]
1


IndexError: tuple index out of range